In [109]:
import numpy as np

# Вспомогательные функции, переводят кубиты в виде |xyz> в векторную форму [a b c ... ]
# И обратно


def kubits_to_vector(kubits_array):
    if not np.array_equal(kubits_array, kubits_array.astype(bool)):
        raise ValueError("Массив кубитов должен содержать только 0 и 1.")

    num_states = 2 ** kubits_array.size
    
    state_vector = np.zeros(num_states)
    
    index = int(''.join(map(str, kubits_array)), 2)
    
    state_vector[index] = 1
    
    return state_vector

def vector_to_kubits(vector):
    if not np.isclose(np.sum(vector), 1) or not np.isin([1], vector):
        raise ValueError("Вектор должен содержать ровно одну единицу.")
    
    index = np.argmax(vector)
    
    num_kubits = int(np.log2(vector.size))
    
    kubits_string = format(index, '0{}b'.format(num_kubits))
    
    kubits_array = np.array([int(bit) for bit in kubits_string])
    
    return kubits_array

In [110]:
import numpy as np
import cirq


class XOR_3(cirq.Gate):
    def __init__(self):
        super(XOR_3, self)

    def _num_qubits_(self):
        return 4

    def _unitary_(self):
        # Для каждого входа определим ожидаемый результат:
        # |0000> - |0000>
        # |0010> - |0011>
        # |0100> - |0101>
        # |0110> - |0110>
        # |1000> - |1001>
        # |1010> - |1010>
        # |1100> - |1100>
        # |1110> - |1111>
        #
        # Для входов, где 4 кубит не равен нулю результат неопределён (не имеет смысла по задаче)
        #
        # Переведём все наборы кубитов в векторный вид:
        # [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] - [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
        # [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0] - [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
        # [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0] - [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
        # [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] - [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
        # [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0] - [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
        # [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0] - [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
        # [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0] - [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
        # [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0] - [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
        #
        # Хотим найти матрицу, которая при умножении на вектор из первой колонки (X) даст вектор из второй колонки (Y)
        # Найдем индекс `x`, под которым в X стоит единица. Строка `x` нашей матрицы должна быть равна вектору Y
        # (исходя из правил перемножения матрицы и вектора)
        #
        # Например, входу |1000> должен соответствовать выход |1001>
        # |1000> в векторной форме будет [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
        # Единица стоит под индексом 8 (неудивительно, 1000 в двоичной равно 8 в десятичной)
        # Значит 8 строка матрицы должна быть |1001> = [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
        # 
        # Таким образом определяем, какими будут строки нашей матрицы с чётными индексами (начиная с нуля)
        #
        # Остальные строки заполним таким образом, чтобы в каждом столбце и каждой строке матрицы была ровно одна единица
        # Это гарантирует, что матрица будет унитарной

        return np.array(
            [
                # 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
                [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # |0000> - |0000>
                [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # |0010> - |0011>
                [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # |0100> - |0101>
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # |0110> - |0110>
                [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # |1000> - |1001>
                [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],  # |1010> - |1010>
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],  # |1100> - |1100>
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],  # |1110> - |1111>
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
            ]
        )

    def _circuit_diagram_info_(self):
        return ["XOR_3"] * self.num_qubits()

In [111]:
def make_cirquit():
    qubits = [cirq.LineQubit(i) for i in range(4)]
    circuit = cirq.Circuit()
    circuit.append(XOR_3().on_each(qubits))
    return circuit


def simulate(circuit, input):
    initial_state = kubits_to_vector(input)

    simulator = cirq.Simulator()
    result = simulator.simulate(circuit, initial_state=initial_state)

    result_kubits = vector_to_kubits(result.final_state_vector)
    print("Input:", input, "Output:", result_kubits)

In [112]:
inputs = [
    np.array([0, 0, 0, 0]),
    np.array([0, 0, 1, 0]),
    np.array([0, 1, 0, 0]),
    np.array([0, 1, 1, 0]),
    np.array([1, 0, 0, 0]),
    np.array([1, 0, 1, 0]),
    np.array([1, 1, 0, 0]),
    np.array([1, 1, 1, 0]),
]

circuit = make_cirquit()

for input in inputs:
    simulate(circuit, input)

Input: [0 0 0 0] Output: [0 0 0 0]
Input: [0 0 1 0] Output: [0 0 1 1]
Input: [0 1 0 0] Output: [0 1 0 1]
Input: [0 1 1 0] Output: [0 1 1 0]
Input: [1 0 0 0] Output: [1 0 0 1]
Input: [1 0 1 0] Output: [1 0 1 0]
Input: [1 1 0 0] Output: [1 1 0 0]
Input: [1 1 1 0] Output: [1 1 1 1]
